In [16]:
import json
import yaml
import uuid
import os

In [13]:
import wombat.bsub as bsub

In [2]:
m = yaml.safe_load(open('../tests/data/align_wxs/HT191P1-S1H1A3Y3.yaml'))
m

{'id': 'HT191P1-S1H1A3Y3',
 'rna-seq': {'tumor': {'R1': {'filepath': '/scratch1/fs1/dinglab/estorrs/pecgs_resources/test_samples/HT191P1-S1H1A3Y3/rna-seq/ht191p1-s1h1a3y3e1_1.AAGGTGTTAG-TAAGCGCGTG/ht191p1-s1h1a3y3e1_1.AAGGTGTTAG-TAAGCGCGTG.HFWJGDSXY_AAGGTGTTAG-TAAGCGCGTG_L001_R1.fastq.gz'},
   'R2': {'filepath': '/scratch1/fs1/dinglab/estorrs/pecgs_resources/test_samples/HT191P1-S1H1A3Y3/rna-seq/ht191p1-s1h1a3y3e1_1.AAGGTGTTAG-TAAGCGCGTG/ht191p1-s1h1a3y3e1_1.AAGGTGTTAG-TAAGCGCGTG.HFWJGDSXY_AAGGTGTTAG-TAAGCGCGTG_L001_R2.fastq.gz'}}},
 'wxs': {'normal': {'R1': {'filepath': '/scratch1/fs1/dinglab/estorrs/pecgs_resources/test_samples/HT191P1-S1H1A3Y3/wxs/CATTATCGCT-CTTGAAGGTT_S34_L001_R1_001.fastq.gz'},
   'R2': {'filepath': '/scratch1/fs1/dinglab/estorrs/pecgs_resources/test_samples/HT191P1-S1H1A3Y3/wxs/CATTATCGCT-CTTGAAGGTT_S34_L001_R2_001.fastq.gz'}},
  'tumor': {'R1': {'filepath': '/scratch1/fs1/dinglab/estorrs/pecgs_resources/test_samples/HT191P1-S1H1A3Y3/wxs/CCAGTAGCGT-ATGTATTGGC_S53

In [6]:
t = yaml.safe_load(open('/home/estorrs/pecgs-pipeline/submodules/align-dnaseq/cwl/template.align_dnaseq.yaml'))
t

{'cpu': 'a_int',
 'flowcell': 'a_string',
 'fq_1': {'class': 'File', 'path': 'a/file/path'},
 'fq_2': {'class': 'File', 'path': 'a/file/path'},
 'index_sequencer': 'a_string',
 'known_sites': {'class': 'File',
  'path': 'path/to/vcf.gz',
  'secondaryFiles': [{'class': 'File', 'path': 'path/to/vcf.gz.tbi'}]},
 'lane': 'a_string',
 'library_preparation': 'a_string',
 'platform': 'a_string',
 'reference': {'class': 'File',
  'path': 'path/to/GRCh38.d1.vd1.fa',
  'secondaryFiles': [{'class': 'File', 'path': 'path/to/GRCh38.d1.vd1.fa.amb'},
   {'class': 'File', 'path': 'path/to/GRCh38.d1.vd1.fa.ann'},
   {'class': 'File', 'path': 'path/to/GRCh38.d1.vd1.fa.bwt'},
   {'class': 'File', 'path': 'path/to/GRCh38.d1.vd1.fa.fai'},
   {'class': 'File', 'path': 'path/to/GRCh38.d1.vd1.fa.pac'},
   {'class': 'File', 'path': 'path/to/GRCh38.d1.vd1.fa.sa'},
   {'class': 'File', 'path': 'path/to/GRCh38.d1.vd1.dict'}]},
 'sample': 'a_string'}

In [7]:
# ID: flowcell.lane
# SM: sample_name
# PL: technology, i.e. ILLUMINA
# PU: flowcell.lane.index_sequencer
# LB: sample_name.library_preparation

In [30]:
template = {
    'fq_1': {
        'class': 'File',
        'path': m['wxs']['normal']['R1']['filepath']
    },
    'fq_2': {
        'class': 'File',
        'path': m['wxs']['normal']['R2']['filepath']
    },
    'known_sites': {
        'class': 'File',
        'path': '/scratch1/fs1/dinglab/estorrs/pecgs_resources/dnaseq_alignment/dbsnp/00-All.chr.vcf.gz',
        'secondaryFiles': [
            {'class': 'File', 'path': '/scratch1/fs1/dinglab/estorrs/pecgs_resources/dnaseq_alignment/dbsnp/00-All.chr.vcf.gz.tbi'}
        ]
    },
    'reference': {
        'class': 'File',
        'path': '/scratch1/fs1/dinglab/estorrs/pecgs_resources/cnv/references/GRCh38.d1.vd1/GRCh38.d1.vd1.fa',
        'secondaryFiles': [
            {'class': 'File', 'path': '/scratch1/fs1/dinglab/estorrs/pecgs_resources/cnv/references/GRCh38.d1.vd1/GRCh38.d1.vd1.fa.amb'},
            {'class': 'File', 'path': '/scratch1/fs1/dinglab/estorrs/pecgs_resources/cnv/references/GRCh38.d1.vd1/GRCh38.d1.vd1.fa.ann'},
            {'class': 'File', 'path': '/scratch1/fs1/dinglab/estorrs/pecgs_resources/cnv/references/GRCh38.d1.vd1/GRCh38.d1.vd1.fa.bwt'},
            {'class': 'File', 'path': '/scratch1/fs1/dinglab/estorrs/pecgs_resources/cnv/references/GRCh38.d1.vd1/GRCh38.d1.vd1.fa.fai'},
            {'class': 'File', 'path': '/scratch1/fs1/dinglab/estorrs/pecgs_resources/cnv/references/GRCh38.d1.vd1/GRCh38.d1.vd1.fa.pac'},
            {'class': 'File', 'path': '/scratch1/fs1/dinglab/estorrs/pecgs_resources/cnv/references/GRCh38.d1.vd1/GRCh38.d1.vd1.fa.sa'},
            {'class': 'File', 'path': '/scratch1/fs1/dinglab/estorrs/pecgs_resources/cnv/references/GRCh38.d1.vd1/GRCh38.d1.vd1.dict'}
        ]
    },
    'flowcell': 'HH7KNDSXY',
    'lane': '1',
    'index_sequencer': 'CATTATCGCT-CTTGAAGGTT',
    'library_preparation': 'TWCE-HT191P1-JM1D1_1-lib1',
    'platform': 'ILLUMINA',
    'sample': m['id'] + '.wxs.N',
    'cpu': 40
}

In [31]:
yaml.safe_dump(template, open('../tests/data/align_dnaseq/HT191P1-S1H1A3Y3_inputs.yaml', 'w'))

In [32]:
workflow_root = '/scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/align_dnaseq/HT191P1-S1H1A3Y3'
output_fp = '../tests/data/align_dnaseq/HT191P1-S1H1A3Y3.cromwell-config-db.compute1.dat'
bsub.save_compute1_cromwell_template(workflow_root, output_fp)

In [33]:
TEST_DIR_ROOT = '/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/align_dnaseq'
TOOL_ROOT = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline'

dconfig = os.path.join(TEST_DIR_ROOT, 'HT191P1-S1H1A3Y3.cromwell-config-db.compute1.dat')
cwl_fp = os.path.join(TOOL_ROOT, 'submodules', 'align-dnaseq', 'cwl', 'align_dnaseq.cwl')
inputs_fp = os.path.join(TEST_DIR_ROOT, 'HT191P1-S1H1A3Y3_inputs.yaml')
volumes = [TEST_DIR_ROOT, TOOL_ROOT, workflow_root, '/storage1/fs1/dinglab/Active', '/storage1/fs1/m.wyczalkowski/Active', '/scratch1/fs1/dinglab/estorrs']

In [34]:
args = bsub.DEFAULT_ARGS
args['group_name'] = 'test_cromwell'
args

{'mem': 10,
 'n_processes': 1,
 'max_mem': None,
 'docker': 'python:3.8',
 'queue': 'dinglab',
 'group': 'compute-dinglab',
 'group_name': 'test_cromwell',
 'n_concurrent': 10,
 'interactive': False,
 'username': 'estorrs'}

In [35]:
start_commands, run_command = bsub.cromwell_commands(dconfig, cwl_fp, inputs_fp, args, volumes, workflow_root=workflow_root)
filepath = os.path.join('../tests/data/align_dnaseq/start_server.sh')
bsub.write_command_file(start_commands, filepath)
filepath = os.path.join('../tests/data/align_dnaseq/run_job.sh')
bsub.write_command_file([run_command], filepath)

In [36]:
for c in start_commands: print(c)

mkdir -p /scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/align_dnaseq/HT191P1-S1H1A3Y3/cromwell-workdir/logs
source /opt/ibm/lsfsuite/lsf/conf/lsf.conf
export LSF_DOCKER_NETWORK=host
export LSF_DOCKER_VOLUMES="/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/align_dnaseq:/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/align_dnaseq /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline:/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline /scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/align_dnaseq/HT191P1-S1H1A3Y3:/scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/align_dnaseq/HT191P1-S1H1A3Y3 /storage1/fs1/dinglab/Active:/storage1/fs1/dinglab/Active /storage1/fs1/m.wyczalkowski/Active:/storage1/fs1/m.wyczalkowski/Active /scratch1/fs1/dinglab/estorrs:/scratch1/fs1/dinglab/estorrs"
bgadd -L 10 /estorrs/test_cromwell
bsub -n 1 -q dinglab-interactive -G compute-dinglab -a 'docker(mwyczalkowski/cromwell-runner)' -g /est

In [37]:
print(run_command)

/usr/bin/java -Dconfig.file=/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/align_dnaseq/HT191P1-S1H1A3Y3.cromwell-config-db.compute1.dat -Djavax.net.ssl.trustStorePassword=changeit -Djavax.net.ssl.trustStore=/gscmnt/gc2560/core/genome/cromwell/cromwell.truststore -jar /usr/local/cromwell/cromwell-47.jar run -t cwl -i /storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/align_dnaseq/HT191P1-S1H1A3Y3_inputs.yaml /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline/submodules/align-dnaseq/cwl/align_dnaseq.cwl
